# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
w_df = pd.read_csv('results.csv')
w_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Constitución,-35.33,-72.42,285.66,92.0,100.0,3.96,CL,1.596333e+09
1,1,Chapais,49.78,-74.85,292.15,88.0,1.0,1.50,CA,1.596333e+09
2,2,Manta,-0.95,-80.73,296.15,83.0,75.0,5.10,EC,1.596333e+09
3,3,Chumikan,54.72,135.31,284.05,90.0,100.0,5.72,RU,1.596333e+09
4,4,Herat Province,34.50,62.00,291.12,36.0,0.0,3.29,AF,1.596333e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [7]:
locations = w_df[["Lat", "Lng"]]
weight = w_df["Humidity"]

In [12]:
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
n_df = w_df[w_df['Wind Speed'] < 3] 

In [18]:
n2_df = n_df[n_df['Humidity'] < 40]
len(n2_df)

27

In [20]:
n3_df = n2_df[n2_df['Max Temp'] > 300]
n3_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,97,Norman Wells,65.28,-126.83,304.15,35.0,20.0,1.50,CA,1.596333e+09
153,164,Riyadh,24.69,46.72,309.15,22.0,88.0,1.50,SA,1.596333e+09
182,198,Iranshahr,27.20,60.68,306.87,35.0,0.0,2.47,IR,1.596333e+09
272,294,Elko,41.00,-115.50,310.93,2.0,50.0,0.45,US,1.596333e+09
278,300,Adrar,20.50,-10.07,307.95,11.0,0.0,1.50,MR,1.596333e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = n3_df
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,97,Norman Wells,65.28,-126.83,304.15,35.0,20.0,1.50,CA,1.596333e+09
153,164,Riyadh,24.69,46.72,309.15,22.0,88.0,1.50,SA,1.596333e+09
182,198,Iranshahr,27.20,60.68,306.87,35.0,0.0,2.47,IR,1.596333e+09
272,294,Elko,41.00,-115.50,310.93,2.0,50.0,0.45,US,1.596333e+09
278,300,Adrar,20.50,-10.07,307.95,11.0,0.0,1.50,MR,1.596333e+09


In [22]:
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\JulianBrown\anaconda3\envs\PythonData1\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
91,97,Norman Wells,65.28,-126.83,304.15,35.0,20.0,1.50,CA,1.596333e+09,
153,164,Riyadh,24.69,46.72,309.15,22.0,88.0,1.50,SA,1.596333e+09,
182,198,Iranshahr,27.20,60.68,306.87,35.0,0.0,2.47,IR,1.596333e+09,
272,294,Elko,41.00,-115.50,310.93,2.0,50.0,0.45,US,1.596333e+09,
278,300,Adrar,20.50,-10.07,307.95,11.0,0.0,1.50,MR,1.596333e+09,
397,427,Korla,41.76,86.15,301.84,15.0,64.0,1.90,CN,1.596333e+09,
456,490,Nūh,28.12,77.02,305.37,5.0,1.0,2.18,IN,1.596334e+09,
560,603,Aksu,41.12,80.26,302.46,21.0,40.0,1.95,CN,1.596334e+09,


In [36]:
for index, row in hotel_df.iterrows():
# geocoordinates 
    lat = row["Lat"]
    lng = row["Lng"]
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

# set up a parameters dictionary
    params = {
        "location": f"{lat},{lng}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = places_data["results"][0]["name"]
    except (IndexError):
        print("Missing field/result... skipping.")
    
hotel_df.head()



Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
91,97,Norman Wells,65.28,-126.83,304.15,35.0,20.0,1.50,CA,1.596333e+09,Heritage Hotel
153,164,Riyadh,24.69,46.72,309.15,22.0,88.0,1.50,SA,1.596333e+09,Four Seasons Hotel Riyadh At Kingdom Center
182,198,Iranshahr,27.20,60.68,306.87,35.0,0.0,2.47,IR,1.596333e+09,هتل و رستوران قصر
272,294,Elko,41.00,-115.50,310.93,2.0,50.0,0.45,US,1.596333e+09,
278,300,Adrar,20.50,-10.07,307.95,11.0,0.0,1.50,MR,1.596333e+09,


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [43]:
hotel_list = hotel_df["Hotel Name"].tolist()
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=["Hotel Name: {Hotel Name}" for hotel in hotel_list])
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

In [44]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))